### IMPORTS

In [1]:
import json
import base64
import hashlib
import subprocess
import sys
import time
import binascii

In [2]:
from web3 import Web3

### FIX FOR WINDOWS ENVIRONMENT

In [3]:
#import pkg_resources.py2_warn

### MIDDLEWARE STARTS

In [4]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [5]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [6]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

In [7]:
print('[X] Read external data!')

[X] Read external data!


### EXTRACT PUBLIC/PRIVATE KEY FROM FILE

In [8]:
def extract(path):
    
    # OPEN THE FILE
    with open(path) as file:
        encrypted_key = file.read()
        
        # DECRYPT & CONVERT FROM HEXBYTES
        private_key = web3.eth.account.decrypt(encrypted_key, 'pass')
        hexed = web3.toHex(private_key)
        
        # PARSE FILE AS JSON & EXTRACT THE ADDRESS
        obj = json.loads(encrypted_key)
        address = '0x' + obj['address']
        
        # GENERATE THE CHECKSUM VERSION
        checksum = web3.toChecksumAddress(address)
        
        return {
            'public': checksum,
            'private': hexed
        }

### CONNECT TO BLOCKCHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + settings['gateway']['host'] + ':' + settings['gateway']['port']))

In [10]:
if web3.isConnected():
    print('[X] Connected with gateway!')
else:
    print('[ ] Could not connect to gateway!')
    exit

[X] Connected with gateway!


### CONSTRUCT SMART CONTRACT INTERFACE

In [23]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': settings['keys']['public'],
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(20, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(settings['keys']['public'])

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = settings['keys']['private']
            )

            # SEND THE TRANSACTION
            tx_hash = web3.eth.sendRawTransaction(signed.rawTransaction)

            # WAIT FOR IT TO BE MINED
            return web3.eth.waitForTransactionReceipt(tx_hash, 500)
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### SERIALIZE NECESSARY MANAGER CONTRACTS

In [12]:
user_manager = contract(latest['usermanager'])

In [13]:
device_manager = contract(latest['devicemanager'])

In [14]:
service_manager = contract(latest['servicemanager'])

In [15]:
task_manager = contract(latest['taskmanager'])

In [16]:
token_manager = contract(latest['tokenmanager'])

In [17]:
print('[X] Manager contracts set!')

[X] Manager contracts set!


### INIT MANAGER CONTRACTS

In [18]:
def init_managers():
    
    # TOKEN PARAMS
    token_symbol = 'ArcaCoin'
    token_price = 5000
    token_capacity = 10000
    
    # TASK PARAMS
    task_fee = 2
    
    # TOKEN MANAGER
    token_manager.write({
        'func': 'init',
        'params': [
            token_symbol,
            token_price,
            token_capacity,
            task_manager.address
        ]
    })
    
    # TASK MANAGER
    task_manager.write({
        'func': 'init',
        'params': [
            task_fee,
            user_manager.address,
            device_manager.address,
            token_manager.address,
            service_manager.address
        ]
    })
    
    # USER MANAGER
    user_manager.write({
        'func': 'init',
        'params': [task_manager.address]
    })
    
    # DEVICE MANAGER
    device_manager.write({
        'func': 'init',
        'params': [
            user_manager.address,
            service_manager.address,
            task_manager.address
        ]
    })
    
    return True

### INITIALIZE MANAGERS IF NECESSARY

In [19]:
def check():
    
    # CHECK STATUS
    initialized = device_manager.read('initialized')
    
    # INIT HAS BEEN PERFORMED
    if (initialized):
        print('[X] Manager contracts have already been initialized!')

    # INIT NOW
    else:
        result = init_managers()

        if (result):
            print('[X] Initializing manager contracts!')

In [20]:
check()

[X] Manager contracts have already been initialized!


### CHECK AGAIN TO CONFIRM

In [21]:
check()

[X] Manager contracts have already been initialized!


task_manager.write({
    'func': 'add',
    'params': [
        'foobar',
        'ea89158adbe59aa9c5e8147c50f2e5cccc3a0d565202d58abc57d5c8946b78e7',
        1,
        'dsfsdfdsfs',
        5
    ]
})